# Fetch Codebase and Models

In [27]:
import os

path = os.path.join(os.getcwd(), 'content')
if not os.path.exists(path):
    os.mkdir(path)
os.chdir(path)

FileNotFoundError: [Errno 2] No such file or directory

In [26]:
CODE_DIR = 'interfacegan'
!git clone https://github.com/genforce/interfacegan.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')
!pwd

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'interfacegan': No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: './interfacegan'

In [ ]:
def wget(path):
    f'wget -P {path}

In [3]:
wget = wget(path)

In [4]:
!wget https://www.dropbox.com/s/t74z87pk3cf8ny7/pggan_celebahq.pth?dl=1 -O models/pretrain/pggan_celebahq.pth --quiet
!wget https://www.dropbox.com/s/nmo2g3u0qt7x70m/stylegan_celebahq.pth?dl=1 -O models/pretrain/stylegan_celebahq.pth --quiet
!wget https://www.dropbox.com/s/qyv37eaobnow7fu/stylegan_ffhq.pth?dl=1 -O models/pretrain/stylegan_ffhq.pth --quiet

In [5]:
!export CUDA_VISIBLE_DEVICES=0

# Import Drive Files (only Colab)

In [1]:
from google.colab import drive

mount_dir = '/content/drive'
dir = mount_dir + '/MyDrive/ONOT-icao-only/ONOT-icao-only'
drive.mount(mount_dir, force_remount = False)

ModuleNotFoundError: No module named 'google'

# Import Dataset (only Local files)

In [17]:
dir = os.path.join(path, 'ONOT-icao-only')
wget = wget(dir)
!wget https://www.dropbox.com/scl/fo/c3c21xyueq4hea0rl9i3r/ADAbxrcnBOFfkywceUhWi_c?rlkey=jk6gd7emcebomf2v4zzgh1ean&st=fkyhp479&dl=0 --quiet

/home/dcaushaj/tir-ONOT/Tir-ONOT/content/interfacegan/interfacegan


# Models parameters

In [7]:
MODEL_POOL2 = {
    'pggan_celebahq': {
        'tf_model_path': '',
        'model_path': dir,
        'gan_type': 'pggan',
        'dataset_name': 'celebahq',
        'latent_space_dim': 512,
        'resolution': 1024,
        'min_val': -1.0,
        'max_val': 1.0,
        'output_channels': 3,
        'channel_order': 'RGB',
        'fused_scale': False,
    },
    'stylegan_celebahq': {
        'tf_model_path':'',
        'model_path': dir,
        'gan_type': 'stylegan',
        'dataset_name': 'celebahq',
        'latent_space_dim': 512,
        'w_space_dim': 512,
        'resolution': 1024,
        'min_val': -1.0,
        'max_val': 1.0,
        'output_channels': 3,
        'channel_order': 'RGB',
        'fused_scale': 'auto',
    },
    'stylegan_ffhq': {
        'tf_model_path': '',
        'model_path': dir,
        'gan_type': 'stylegan',
        'dataset_name': 'ffhq',
        'latent_space_dim': 512,
        'w_space_dim': 512,
        'resolution': 1024,
        'min_val': -1.0,
        'max_val': 1.0,
        'output_channels': 3,
        'channel_order': 'RGB',
        'fused_scale': 'auto',
    },
}

# Functions

In [8]:
import os.path
import io
import IPython.display
import numpy as np
import cv2 as cv
import PIL.Image

import torch

from models.pggan_generator import PGGANGenerator
from models.stylegan_generator import StyleGANGenerator
from utils.manipulator import linear_interpolate

def load_images_from_folder(folder):
  images = []
  if not os.path.exists(folder):
      raise Exception(f'{folder} does not exist')
  for filename in os.listdir(folder):
      filepath = os.path.join(folder, filename)
      img = cv.imread(filepath)
      if img is not None:
          images.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
  if len(images) == 0:
      raise Exception(f"No images found in {folder}")
  return np.array(images)

def build_generator(model_name):
  """Builds the generator by model name."""
  gan_type = MODEL_POOL2[model_name]['gan_type']
  if gan_type == 'pggan':
    generator = PGGANGenerator(model_name)
  elif gan_type == 'stylegan':
    generator = StyleGANGenerator(model_name)
  return generator


def sample_codes(generator, num, latent_space_type='Z', seed=0):
  """Samples latent codes randomly."""
  np.random.seed(seed)
  codes = generator.easy_sample(num)
  if generator.gan_type == 'stylegan' and latent_space_type == 'W':
    codes = torch.from_numpy(codes).type(torch.FloatTensor).to(generator.run_device)
    codes = generator.get_value(generator.model.mapping(codes))
  return codes


def imshow(images, col, viz_size=256):
    """Shows images in one figure."""
    num, height, width, channels = images.shape
    assert num % col == 0, "num % col != 0"
    row = num // col

    fused_image = np.zeros((viz_size * row, viz_size * col, channels), dtype=np.uint8)

    for idx, image in enumerate(images):
        i, j = divmod(idx, col)
        y = i * viz_size
        x = j * viz_size
        if height != viz_size or width != viz_size:
            image = cv.resize(image, (viz_size, viz_size))
        fused_image[y:y + viz_size, x:x + viz_size] = image

    fused_image = np.asarray(fused_image, dtype=np.uint8)
    data = io.BytesIO()
    PIL.Image.fromarray(fused_image).save(data, 'jpeg')
    im_data = data.getvalue()
    disp = IPython.display.display(IPython.display.Image(im_data))
    return disp

def load_images_as_latent_codes(generator):
    latent_codes = sample_codes(generator, len(images), latent_space_type, noise_seed)
    print(noise_seed)
    return latent_codes, images

def load_boundaries(model_name, latent_space_type):
    ATTRS = ['age', 'eyeglasses', 'gender', 'pose', 'smile']
    boundaries = {}
    for attr_name in ATTRS:
        boundary_name = f'{model_name}_{attr_name}'
        if generator.gan_type == 'stylegan' and latent_space_type == 'W':
            boundaries[attr_name] = np.load(f'boundaries/{boundary_name}_w_boundary.npy')
        else:
            boundaries[attr_name] = np.load(f'boundaries/{boundary_name}_boundary.npy')
    return boundaries

# Select a model

In [9]:
#@title { display-mode: "form", run: "auto" }
model_name = "pggan_celebahq" #@param ['pggan_celebahq','stylegan_celebahq', 'stylegan_ffhq']
latent_space_type = "W" #@param ['Z', 'W']

generator = build_generator(model_name)
boundaries = load_boundaries(model_name, latent_space_type)

[2024-07-22 15:49:18,357][INFO] Loading pytorch model from `models/pretrain/pggan_celebahq.pth`.
[2024-07-22 15:49:18,441][INFO] Successfully loaded!
[2024-07-22 15:49:18,443][INFO]   `lod` of the loaded model is 0.0.


# Load Images

In [10]:
images = load_images_from_folder(dir)

Exception: /home/dcaushaj/tir-ONOT/Tir-ONOT/content/ONOT-icao-only does not exist

# Sample latent codes

In [ ]:
#@title { display-mode: "form", run: "auto" }

num_samples = 4 #@param {type:"slider", min:1, max:8, step:1}
noise_seed = 2 #@param {type:"slider", min:0, max:87, step:1}

latent_codes, sampled_images = load_images_as_latent_codes(generator)
#firstL = latent_codes[noise_seed*num_samples:noise_seed*num_samples+num_samples]
imshow(sampled_images[noise_seed*num_samples:noise_seed*num_samples+num_samples], col=num_samples)

# Edit facial attributes

In [ ]:
#@title { display-mode: "form", run: "auto" }

age = 0.5 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
eyeglasses = 3 #@param {type:"slider", min:-2.9, max:3.0, step:0.1}
gender = -2.1 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
pose = -2.1 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
smile = 0.9 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}

modifications = {
        'age': age,
        'eyeglasses': eyeglasses,
        'gender': gender,
        'pose': pose,
        'smile': smile
    }

new_codes = latent_codes.copy()
#secL = new_codes[noise_seed*num_samples:noise_seed*num_samples+num_samples]
#print(firstL == secL)

for attr_name, value in modifications.items():
    new_codes += boundaries[attr_name] * value

if generator.gan_type == 'stylegan' and latent_space_type == 'W':
    synthesis_kwargs = {'latent_space_type': 'W'}
else:
    synthesis_kwargs = {}

# Synthesize images with modified latent codes
new_images = generator.easy_synthesize(new_codes[noise_seed*num_samples:noise_seed*num_samples+num_samples],
                                       **synthesis_kwargs)['image']

imshow(new_images, col=len(new_images))